In [2]:
import ROOT as r

%jsroot on

# Checks with Full Simulation

In [3]:
folder = '/disk/lhcb_data2/RLcMuonic2016/MC_full_trueTrigger/'
mcsamples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']
variables = {'FitVar_El_mLc':['12','0','2600','E_{#mu}'],'FitVar_q2_mLc/1.E6':['4','-2','14','q^{2}'],
             'FitVar_Mmiss2_mLc/1.E6':['12','-2','14','M_{miss}^{2}']}

In [4]:
def PutTogetherPolarityHistos(h):
    h_new = {}
    for mcsample in mcsamples:
        if(h[mcsample]['MagUp']):
            h_new[mcsample] = h[mcsample]['MagUp']
            if(h[mcsample]['MagDown']):
                h_new[mcsample].Add(h[mcsample]['MagDown'])
                h_new[mcsample].SetTitle(mcsample)
                h_new[mcsample].SetDirectory(0)
    return h_new

In [5]:
def PlotSuperimposed(var,htrue,hemul,Xaxisname, Yaxisname):
    c = r.TCanvas('c_'+var,'',800,1500)
    nsamples = len(mcsamples)
    npads = int(nsamples/2+1)
    c.Divide(2,npads)
    for i,mcsample in enumerate(mcsamples):
        c.cd(i+1)
        htrue[mcsample].SetLineColor(r.kAzure+9)
        htrue[mcsample].GetXaxis().SetTitle(Xaxisname)
        hemul[mcsample].SetLineColor(r.kOrange+2)
        htrue[mcsample].GetYaxis().SetTitle(Yaxisname)
        htrue[mcsample].Draw('hist')
        hemul[mcsample].Draw('hist sames')
        l = r.TLegend(0.5,0.6,0.8,0.85)
        l.AddEntry(htrue[mcsample],'True Trigger','l')
        l.AddEntry(hemul[mcsample],'Emulated Trigger','l')
        l.Draw('sames')
    return c

In [6]:
def CompareL0(var,nbins,bmin,bmax):
    htrue = {mcsample: {polarity: r.TH1F('htrue_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    hemul = {mcsample: {polarity: r.TH1F('hemul_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    hnocut = {mcsample: {polarity: r.TH1F('hnocut_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    for mcsample in mcsamples:
        for polarity in polarities:
            startf = r.TFile(folder+mcsample+'_'+polarity+'_full.root','READ')
            startt = startf.Get('tupleout/DecayTree')
            preself = r.TFile(folder+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
            preselt = preself.Get('DecayTree')
            startt.AddFriend(preselt)
            ftrig = r.TFile(folder+mcsample+'_'+polarity+'_full_wL0TOSEmulation.root','READ')
            ttrig = ftrig.Get('DecayTree')
            ftrig1 = r.TFile(folder+mcsample+'_'+polarity+'_full_wL0TISEmulation.root','READ')
            ttrig1 = ftrig1.Get('DecayTree')
            startt.AddFriend(ttrig)
            startt.AddFriend(ttrig1)
            name = mcsample+'_'+polarity
            startt.Draw(var+'>>'+var+'_true_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_L0HadronDecision_TOS==1||Lb_L0Global_TIS==1)&&(TruthMatch==1&&MoreHLT2Sel==1&&nSPDHits<450)')
            htrue[mcsample][polarity] = r.gPad.GetPrimitive(var+'_true_'+name)
            startt.Draw(var+'>>'+var+'_emul_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_L0Hadron_TOS_emulated==1||Lb_L0Global_TIS_emulated==1)&&(TruthMatch==1&&MoreHLT2Sel==1&&nSPDHits<450)')
            hemul[mcsample][polarity] = r.gPad.GetPrimitive(var+'_emul_'+name)
            startt.Draw(var+'>>'+var+'_nocut_'+name+'('+nbins+','+bmin+','+bmax+')',
                        'TruthMatch==1&&MoreHLT2Sel==1&&nSPDHits<450')
            hnocut[mcsample][polarity] = r.gPad.GetPrimitive(var+'_nocut_'+name)
            htrue[mcsample][polarity].SetDirectory(0)
            hemul[mcsample][polarity].SetDirectory(0)
            hnocut[mcsample][polarity].SetDirectory(0)
    
    htrue_1 = PutTogetherPolarityHistos(htrue)
    hemul_1 = PutTogetherPolarityHistos(hemul)
    hnocut_1 = PutTogetherPolarityHistos(hnocut)
    return htrue_1,hemul_1,hnocut_1

In [7]:
def CompareHLT1(var,nbins,bmin,bmax):
    htrue = {mcsample: {polarity: r.TH1F('htrue_'+var+'_'+mcsample+'_'+polarity,'',40,-2,12) 
                          for polarity in polarities} for mcsample in mcsamples}
    hemul = {mcsample: {polarity: r.TH1F('hemul_'+var+'_'+mcsample+'_'+polarity,'',40,-2,12) 
                          for polarity in polarities} for mcsample in mcsamples}
    hnocut = {mcsample: {polarity: r.TH1F('hnocut_'+var+'_'+mcsample+'_'+polarity,'',40,-2,12) 
                          for polarity in polarities} for mcsample in mcsamples}
    L0cut = '(Lb_L0Global_TIS==1||Lc_L0HadronDecision_TOS == 1)'
    Truthcut = 'MoreHLT2Sel==1&&TruthMatch==1&&nSPDHits<450'
    cut = Truthcut + '&& '+L0cut
    for mcsample in mcsamples:
        for polarity in polarities:
            startf = r.TFile(folder+mcsample+'_'+polarity+'_full.root','READ')
            startt = startf.Get('tupleout/DecayTree')
            preself = r.TFile(folder+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
            preselt = preself.Get('DecayTree')
            startt.AddFriend(preselt)
            ftrig = r.TFile(folder+mcsample+'_'+polarity+'_full_wHLT1OneTrackEmulation_Df.root','READ')
            ttrig = ftrig.Get('DecayTree')
            startt.AddFriend(ttrig)
            ftrig2 = r.TFile(folder+mcsample+'_'+polarity+'_full_wHLT1TwoTracksEmulation.root','READ')
            ttrig2 = ftrig2.Get('DecayTree')
            startt.AddFriend(ttrig2)
            name = mcsample+'_'+polarity
            startt.Draw(var+'>>'+var+'_true_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_Hlt1TrackMVADecision_TOS==1||Lc_Hlt1TwoTrackMVADecision_TOS==1)&&'+cut)
            htrue[mcsample][polarity] = r.gPad.GetPrimitive(var+'_true_'+name)
            startt.Draw(var+'>>'+var+'_emul_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_HLT1TrackMVA_Emu_EffCorrected_TOS==1||Lc_HLT1TwoTrackMVA_Emu_EffCorr_TOS==1)&&'+cut)
            hemul[mcsample][polarity] = r.gPad.GetPrimitive(var+'_emul_'+name)
            startt.Draw(var+'>>'+var+'_nocut_'+name+'('+nbins+','+bmin+','+bmax+')',cut)
            hnocut[mcsample][polarity] = r.gPad.GetPrimitive(var+'_nocut_'+name)
            htrue[mcsample][polarity].SetDirectory(0)
            hemul[mcsample][polarity].SetDirectory(0)
            hnocut[mcsample][polarity].SetDirectory(0)
    
    htrue_1 = PutTogetherPolarityHistos(htrue)
    hemul_1 = PutTogetherPolarityHistos(hemul)
    hnocut_1 = PutTogetherPolarityHistos(hnocut)
    return htrue_1,hemul_1,hnocut_1

### Shapes after L0 trigger

In [8]:
h_L0_true = {}
h_L0_emul = {}
h_tot = {}
for key in variables.keys():
    print(key)
    h_L0_true[key], h_L0_emul[key], h_tot[key] = CompareL0(key, variables[key][0],variables[key][1],
                                                                      variables[key][2])

FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [9]:
cL0 = {}
for key in variables.keys():
    cL0[key] = PlotSuperimposed(key+'_L0_1',h_L0_true[key],h_L0_emul[key],variables[key][3],'')
    cL0[key].Draw()
    name = key
    if(key=='FitVar_q2_mLc/1.E6'): name = "FitVar_q2_mLc"
    if(key=='FitVar_Mmiss2_mLc/1.E6'): name = 'FitVar_Mmiss2_mLc'
    cL0[key].SaveAs('plots/TriggerEmulationShapes/'+name+'_L0.png')

Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_El_mLc_L0.png has been created
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_q2_mLc_L0.png has been created
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_Mmiss2_mLc_L0.png has been created


### Shapes after HLT1 trigger

In [13]:
h_HLT1_true = {}
h_HLT1_emul = {}
h_tot = {}
for key in variables.keys():
    print(key)
    h_HLT1_true[key], h_HLT1_emul[key], h_tot[key] = CompareHLT1(key, variables[key][0],variables[key][1],
                                                                 variables[key][2])

FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [17]:
cHLT1 = {}
for key in variables.keys():
    cHLT1[key] = PlotSuperimposed(key+'_HLT1',h_HLT1_true[key],h_HLT1_emul[key],variables[key][3],'')
    cHLT1[key].Draw()
    name = key
    if(key=='FitVar_q2_mLc/1.E6'): name = "FitVar_q2_mLc"
    if(key=='FitVar_Mmiss2_mLc/1.E6'): name = 'FitVar_Mmiss2_mLc'
    cHLT1[key].SaveAs('plots/TriggerEmulationShapes/'+name+'_HLT1.png')

Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_El_mLc_HLT1.png has been created
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_q2_mLc_HLT1.png has been created
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_Mmiss2_mLc_HLT1.png has been created


# Checks with TrackerOnly samples

In [10]:
folderFull = '/disk/lhcb_data2/RLcMuonic2016/MC_full_trueTrigger/'
folderTO = '/disk/lhcb_data2/RLcMuonic2016/MC_TrackerOnly/'
mcsamples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']
variables = {'FitVar_El_mLc':['12','0','2600','E_{#mu}'],'FitVar_q2_mLc/1.E6':['4','-2','14','q^{2}'],
             'FitVar_Mmiss2_mLc/1.E6':['12','-2','14','M_{miss}^{2}']}

In [11]:
def ScaleHistos(h_emul, h_true, value):
    for key in variables.keys():
        print(key)
        for mcsample in mcsamples:
            scale = value/h_emul[key][mcsample].Integral()
            h_emul[key][mcsample].Scale(scale)
            scale2 = value/h_true[key][mcsample].Integral()
            h_true[key][mcsample].Scale(scale2)
    return h_emul, h_true
    

In [12]:
def CompareL0forTO(var,nbins,bmin,bmax):
    htrue = {mcsample: {polarity: r.TH1F('htrue_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    hemul = {mcsample: {polarity: r.TH1F('hemul_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    branchesON = ['FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc','nSPDHits','Lc_L0HadronDecision_TOS',
                  'Lb_L0Global_TIS']
    for mcsample in mcsamples:
        for polarity in polarities:
            ffull = r.TFile(folderFull+mcsample+'_'+polarity+'_full.root','READ')
            tfull = ffull.Get('tupleout/DecayTree')
            tfull.SetBranchStatus('*',0)
            for br in branchesON:
                tfull.SetBranchStatus(br,1)
            preselffull = r.TFile(folderFull+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
            preseltfull = preselffull.Get('DecayTree')
            tfull.AddFriend(preseltfull)
            fTO = r.TFile(folderTO+mcsample+'_'+polarity+'.root','READ')
            tTO = fTO.Get('tupleout/DecayTree')
            tTO.SetBranchStatus('*',0)
            for br in branchesON:
                if br not in ['Lc_L0HadronDecision_TOS','Lb_L0Global_TIS']:
                    tTO.SetBranchStatus(br,1)
            preselfTO = r.TFile(folderTO+mcsample+'_'+polarity+'_preselectionVars.root','READ')
            preseltTO = preselfTO.Get('DecayTree')
            ftrig = r.TFile(folderTO+mcsample+'_'+polarity+'_wL0TOSEmulation.root','READ')
            ttrig = ftrig.Get('DecayTree')
            ftrig1 = r.TFile(folderTO+mcsample+'_'+polarity+'_wL0TISEmulation.root','READ')
            ttrig1 = ftrig1.Get('DecayTree')
            tTO.AddFriend(preseltTO)
            tTO.AddFriend(ttrig)
            tTO.AddFriend(ttrig1)
            name = mcsample+'_'+polarity
            tfull.Draw(var+'>>'+var+'_true_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_L0HadronDecision_TOS==1||Lb_L0Global_TIS==1)&&(TruthMatch==1&&MoreHLT2Sel==1&&nSPDHits<450)')
            htrue[mcsample][polarity] = r.gPad.GetPrimitive(var+'_true_'+name)
            tTO.Draw(var+'>>'+var+'_emul_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_L0Hadron_TOS_emulated==1||Lb_L0Global_TIS_emulated==1)&&(TruthMatch==1&&MoreHLT2Sel==1&&nSPDHits<450)')
            hemul[mcsample][polarity] = r.gPad.GetPrimitive(var+'_emul_'+name)
            htrue[mcsample][polarity].SetDirectory(0)
            hemul[mcsample][polarity].SetDirectory(0)
            
    
    htrue_1 = PutTogetherPolarityHistos(htrue)
    hemul_1 = PutTogetherPolarityHistos(hemul)
    return htrue_1,hemul_1

In [13]:
h_L0_true = {}
h_L0_emul = {}
for key in variables.keys():
    print(key)
    h_L0_true[key], h_L0_emul[key] = CompareL0forTO(key, variables[key][0],variables[key][1],variables[key][2])

FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [14]:
h_L0_true_scaled, h_L0_emul_scaled = ScaleHistos(h_L0_emul, h_L0_true, 1)

FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [15]:
cL0TO = {}
for key in variables.keys():
    cL0TO[key] = PlotSuperimposed(key+'_L0_1',h_L0_true_scaled[key],h_L0_emul_scaled[key],variables[key][3],'')
    cL0TO[key].Draw()
    name = key
    if(key=='FitVar_q2_mLc/1.E6'): name = "FitVar_q2_mLc"
    if(key=='FitVar_Mmiss2_mLc/1.E6'): name = 'FitVar_Mmiss2_mLc'
    cL0TO[key].SaveAs('plots/TriggerEmulationShapes/'+name+'_L0_TO.png')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_FitVar_El_mLc_L0_1
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_El_mLc_L0_TO.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_FitVar_q2_mLc/1.E6_L0_1
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_q2_mLc_L0_TO.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_FitVar_Mmiss2_mLc/1.E6_L0_1
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_Mmiss2_mLc_L0_TO.png has been created


In [16]:
def CompareHLT1forTO(var,nbins,bmin,bmax):
    htrue = {mcsample: {polarity: r.TH1F('htrue1_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    hemul = {mcsample: {polarity: r.TH1F('hemul1_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    branchesON = ['FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc','nSPDHits','Lc_L0HadronDecision_TOS','Lb_L0Global_TIS','Lc_Hlt1TrackMVADecision_TOS','Lc_Hlt1TwoTrackMVADecision_TOS']
    L0cutFull = '(Lb_L0Global_TIS==1||Lc_L0HadronDecision_TOS == 1)'
    Truthcut = 'MoreHLT2Sel==1&&TruthMatch==1&&nSPDHits<450'
    cutFull = Truthcut + '&& '+L0cutFull
    L0cutTO = '(Lc_L0Hadron_TOS_emulated==1||Lb_L0Global_TIS_emulated==1)'
    cutTO = Truthcut + '&& '+L0cutTO
    for mcsample in mcsamples:
        for polarity in polarities:
            ffull = r.TFile(folderFull+mcsample+'_'+polarity+'_full.root','READ')
            tfull = ffull.Get('tupleout/DecayTree')
            tfull.SetBranchStatus('*',0)
            for br in branchesON:
                tfull.SetBranchStatus(br,1)
            preselffull = r.TFile(folderFull+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
            preseltfull = preselffull.Get('DecayTree')
            tfull.AddFriend(preseltfull)
            
            fTO = r.TFile(folderTO+mcsample+'_'+polarity+'.root','READ')
            tTO = fTO.Get('tupleout/DecayTree')
            tTO.SetBranchStatus('*',0)
            for br in branchesON:
                if br not in ['Lc_L0HadronDecision_TOS','Lb_L0Global_TIS',
                              'Lc_Hlt1TrackMVADecision_TOS','Lc_Hlt1TwoTrackMVADecision_TOS']:
                    tTO.SetBranchStatus(br,1)
            preselfTO = r.TFile(folderTO+mcsample+'_'+polarity+'_preselectionVars.root','READ')
            preseltTO = preselfTO.Get('DecayTree')
            ftrigA = r.TFile(folderTO+mcsample+'_'+polarity+'_wL0TOSEmulation.root','READ')
            ttrigA = ftrigA.Get('DecayTree')
            ftrig1A = r.TFile(folderTO+mcsample+'_'+polarity+'_wL0TISEmulation.root','READ')
            ttrig1A = ftrig1A.Get('DecayTree')
            ftrig = r.TFile(folderTO+mcsample+'_'+polarity+'_wHLT1Emulation.root','READ')
            ttrig = ftrig.Get('DecayTree')
            #ftrig = r.TFile(folderTO+mcsample+'_'+polarity+'_wHLT1OneTrackEmulation_Df.root','READ')
            #ttrig = ftrig.Get('DecayTree')
            #ftrig1 = r.TFile(folderTO+mcsample+'_'+polarity+'_wHLT1TwoTracksEmulation.root','READ')
            #ttrig1 = ftrig1.Get('DecayTree')
            tTO.AddFriend(preseltTO)
            tTO.AddFriend(ttrig)
            #tTO.AddFriend(ttrig1)
            tTO.AddFriend(ttrigA)
            tTO.AddFriend(ttrig1A)
            name = mcsample+'_'+polarity
            tfull.Draw(var+'>>'+var+'_true_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_Hlt1TrackMVADecision_TOS==1||Lc_Hlt1TwoTrackMVADecision_TOS==1)&&'+cutFull)
            htrue[mcsample][polarity] = r.gPad.GetPrimitive(var+'_true_'+name)
            tTO.Draw(var+'>>'+var+'_emul_'+name+'('+nbins+','+bmin+','+bmax+')',
                        '(Lc_HLT1TrackMVA_Emu_EffCorrected_TOS==1||Lc_HLT1TwoTrackMVA_Emu_EffCorr_TOS==1)&&'+cutTO)
            hemul[mcsample][polarity] = r.gPad.GetPrimitive(var+'_emul_'+name)
            htrue[mcsample][polarity].SetDirectory(0)
            hemul[mcsample][polarity].SetDirectory(0)
            
    
    htrue_1 = PutTogetherPolarityHistos(htrue)
    hemul_1 = PutTogetherPolarityHistos(hemul)
    return htrue_1,hemul_1

In [17]:
h_HLT1_true = {}
h_HLT1_emul = {}
for key in variables.keys():
    print(key)
    h_HLT1_true[key], h_HLT1_emul[key] = CompareHLT1forTO(key, variables[key][0],variables[key][1],variables[key][2])

FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [18]:
h_HLT1_true_scaled, h_HLT1_emul_scaled = ScaleHistos(h_HLT1_emul, h_HLT1_true, 1)

FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [19]:
cHLT1TO = {}
for key in variables.keys():
    cHLT1TO[key] = PlotSuperimposed(key+'_HLT1',h_HLT1_true_scaled[key],h_HLT1_emul_scaled[key],variables[key][3],'')
    cHLT1TO[key].Draw()
    name = key
    if(key=='FitVar_q2_mLc/1.E6'): name = "FitVar_q2_mLc"
    if(key=='FitVar_Mmiss2_mLc/1.E6'): name = 'FitVar_Mmiss2_mLc'
    cHLT1TO[key].SaveAs('plots/TriggerEmulationShapes/'+name+'_HLT1_TO.png')

Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_El_mLc_HLT1_TO.png has been created
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_q2_mLc_HLT1_TO.png has been created
Info in <TCanvas::Print>: png file plots/TriggerEmulationShapes/FitVar_Mmiss2_mLc_HLT1_TO.png has been created
